# Notes

## Setup notes
- If installing TTS package on a venv, install propper cuda enabled torch otherwise default torch will be installed, preventing cuda from being used.
- Go to "TTS\tts\layers\tortoise\arch_utils.py" replace references of LogitWarper to LogitsProcessor
- Go to "TTS\tts\models\xtts.py then to function get_compatible_checkpoint_state_dict. On line 714: checkpoint = load_fsspec(model_path, map_location=torch.device("cpu"))["model"]. Add the argument 'weights_only = False": checkpoint = load_fsspec(model_path, map_location=torch.device("cpu"), weights_only = False)["model"]


In [1]:
'''Imports'''
from trainer import Trainer, TrainerArgs
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager
import torch
import sys
import os
from datetime import datetime
import wandb
from trainer.logging.wandb_logger import WandbLogger

C:\Users\12017\Desktop\NJIT\DS677_852_Project\src\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
'''Display device used'''
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
print(torch.version.cuda)           
print(torch.cuda.is_available())    
print(torch.cuda.get_device_name()) 

cuda:0
12.4
True
NVIDIA GeForce RTX 2070


In [3]:
'''DOWNLOADS'''
# Get XTTS files
CHECKPOINT_PATH = './XTTS-files/'
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# DVAE files
DVAE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINT_PATH, os.path.basename(DVAE_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINT_PATH, os.path.basename(MEL_NORM_LINK))

# DVAE download if not exists
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_LINK], CHECKPOINT_PATH, progress_bar=True)

# XTTS v2.0 checkpoint
TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

# Transfer learning parameters. NOTE: Sets base model to use
TOKENIZER_FILE = os.path.join(CHECKPOINT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json
XTTS_CHECKPOINT = os.path.join(CHECKPOINT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth

# XTTS v2.0 download if not exists
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINT_PATH, progress_bar=True
    )
print("Paths set.")

Paths set.


In [5]:
'''DATA LOADING'''
# Set lang
LANGUAGE ='en'
# Set to folder name that contains metadata.csv and wavs dir (with the .wav examples)
DATASET= "noramlized_personal_voice"
training_dir = f'./datasets/{DATASET}/' # change to folder w/ training examples

# Dataset uses ljspeech format
dataset_config = BaseDatasetConfig(
    formatter="ljspeech",
    meta_file_train="metadata.csv", # metadata file w/ transcriptions
    language=LANGUAGE,
    path=training_dir
)

# Turn off eval split. Will evaluate manually
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_size=0.02 # Might change
)


'''MODIFY'''
# Audio config
audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000) 

# Speaker Reference: Match theses to the test sentences
### Only need 1 speaker audio reference. Do not need to match voice to text
SPEAKER_TEXT = [
"It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent.",
    "This cake is great. It's so delicious and moist."
]
SPEAKER_REFERENCE = f"datasets/{DATASET}/wavs/chunk_0016.wav"


 | > Found 703 files in C:\Users\12017\Desktop\NJIT\DS677_852_Project\src\datasets\noramlized_personal_voice


In [6]:
'''Set Model arguments'''
model_args = GPTArgs(
    max_conditioning_length=(22050*11), # Use sample rate units to define max condition length (for reference audio wav)
    min_conditioning_length=(22050*3),
    debug_loading_failures=True,
    max_wav_length=(22050*11), # max 10 seconds
    max_text_length=(22050*3), # min 3 seconds
    mel_norm_file=MEL_NORM_FILE,
    dvae_checkpoint=DVAE_CHECKPOINT,
    xtts_checkpoint=XTTS_CHECKPOINT,  
    tokenizer_file=TOKENIZER_FILE,
    gpt_num_audio_tokens=1026, 
    gpt_start_audio_token=1024,
    gpt_stop_audio_token=1025,
    gpt_use_masking_gt_prompt_approach=True,
    gpt_use_perceiver_resampler=True,
)

In [7]:
'''Set up configuration file'''
'''TRAINING CONFIG'''
OUT_PATH = './training_outputs/'

RUN_NAME = f"xttsv2_finetune_{datetime.now().strftime('%Y%m%d_%H%M')}"
PROJECT_NAME = "XTTS-v2 Finetune"
DASHBOARD_LOGGER = 'wandb'
LOGGER_URI = None

OPTIMIZER_WD_ONLY_ON_WEIGHTS = True  

BATCH_SIZE = 3 # 4 is common

config = GPTTrainerConfig(
    run_eval=True,
    epochs = 10, # assuming you want to end training manually w/ keyboard interrupt
    output_path=OUT_PATH,
    model_args=model_args,
    run_name=RUN_NAME,
    project_name=PROJECT_NAME,
    run_description="""
        GPT XTTS training
        """,
    dashboard_logger=DASHBOARD_LOGGER,
    wandb_entity=None,
    logger_uri=LOGGER_URI,
    audio=audio_config,
    batch_size=BATCH_SIZE,
    batch_group_size=48,
    eval_batch_size=BATCH_SIZE,
    num_loader_workers=0, # On Windows, num_loader_workers > 0 can break multiprocessing in PyTorch
    eval_split_max_size=256, 
    print_step=50, 
    plot_step=100, 
    log_model_step=1000, 
    save_step=1000, # Needs to be an int
    save_n_checkpoints=3, # Rotate last 3 checkpoints
    save_checkpoints=True,
    print_eval=True,
    optimizer="AdamW",
    optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
    optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
    lr=5e-06,  
    lr_scheduler="MultiStepLR",
    lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
    test_sentences=[ 
        {
            "text": SPEAKER_TEXT[0],
            "speaker_wav": SPEAKER_REFERENCE, 
            "language": LANGUAGE,
        },
        {
            "text": SPEAKER_TEXT[1],
            "speaker_wav": SPEAKER_REFERENCE,
            "language": LANGUAGE,
        }
    ],
) 

In [8]:
'''Set up Trainer'''
# Init model 
model = GPTTrainer.init_from_config(config)

# Init Trainer
GRAD_ACUMM_STEPS = 84 # Note: GRAD_ACUMM_STEPS * BATCH_SIZE = 252
START_WITH_EVAL = True  

trainer = Trainer(
    TrainerArgs(
        restore_path=None, # Change to model path if resuming
        skip_train_epoch=False,
        start_with_eval=START_WITH_EVAL,
        grad_accum_steps=GRAD_ACUMM_STEPS,
    ),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 16
 | > Num. of Torch Threads: 1
 | > Torch seed: 1
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


>> DVAE weights restored from: ./XTTS-files/dvae.pth


wandb: Currently logged in as: sliverwall (sliverwall-new-jersey-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



 > Model has 518442047 parameters


In [9]:
'''TRAINING: manual interupts will set model to output saves at given checkpoints'''
trainer.fit()


 > EPOCH: 0/10
 --> ./training_outputs/xttsv2_finetune_20250504_1250-May-04-2025_12+50PM-ca1939c

 > EVALUATION 



 > Filtering invalid eval samples!!
 > Total eval samples after filtering: 14


   --> STEP: 0
     | > loss_text_ce: 0.021762222051620483  (0.021762222051620483)
     | > loss_mel_ce: 3.8056249618530273  (3.8056249618530273)
     | > loss: 3.8273870944976807  (3.8273870944976807)

   --> STEP: 1
     | > loss_text_ce: 0.021934326738119125  (0.021934326738119125)
     | > loss_mel_ce: 3.791006326675415  (3.791006326675415)
     | > loss: 3.8129405975341797  (3.8129405975341797)

   --> STEP: 2
     | > loss_text_ce: 0.02519591525197029  (0.023565120995044708)
     | > loss_mel_ce: 4.018796443939209  (3.904901385307312)
     | > loss: 4.043992519378662  (3.928466558456421)

   --> STEP: 3
     | > loss_text_ce: 0.021754387766122818  (0.022961543252070744)
     | > loss_mel_ce: 3.4101994037628174  (3.7400007247924805)
     | > loss: 3.4319539070129395  (3.7629623413085938)

   --> STEP: 4
     | > loss_text_ce: 0.0275751780718565  (0.024114951957017183)
     | > loss_mel_ce: 4.1670823097229  (3.8467711210250854)
     | > loss: 4.194657325744629  (3.8708860874176025)

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0227736234664917 (+0)
     | > avg_loss_text_ce: 0.024114951957017183 (+0)
     | > avg_loss_mel_ce: 3.8467711210250854 (+0)
     | > avg_loss: 3.8708860874176025 (+0)


 > EPOCH: 1/10
 --> ./training_outputs/xttsv2_finetune_20250504_1250-May-04-2025_12+50PM-ca1939c

 > TRAINING (2025-05-04 12:50:30) 


...

Ignoring sample ./datasets/noramlized_personal_voice/wavs\chunk_0365.wav because it was already ignored before !!
Ignoring sample ./datasets/noramlized_personal_voice/wavs\chunk_0658.wav because it was already ignored before !!
Ignoring sample ./datasets/noramlized_personal_voice/wavs\chunk_0658.wav because it was already ignored before !!



   --> TIME: 2025-05-04 20:50:19 -- STEP: 210/230 -- GLOBAL_STEP: 2050
     | > loss_text_ce: 0.02499692142009735  (0.02472384764502445)
     | > loss_mel_ce: 3.545748710632324  (3.39621425583249)
     | > loss: 0.04250887781381607  (0.04072545429780367)
     | > current_lr: 5e-06 
     | > step_time: 11.4648  (6.09997319493975)
     | > loader_time: 0.0159  (0.019097761880783815)


 > EVALUATION 

   --> STEP: 0
     | > loss_text_ce: 0.020889604464173317  (0.020889604464173317)
     | > loss_mel_ce: 3.302748441696167  (3.302748441696167)
     | > loss: 3.3236379623413086  (3.3236379623413086)

   --> STEP: 1
     | > loss_text_ce: 0.021463457494974136  (0.021463457494974136)
     | > loss_mel_ce: 3.26389217376709  (3.26389217376709)
     | > loss: 3.285355567932129  (3.285355567932129)

   --> STEP: 2
     | > loss_text_ce: 0.024410953745245934  (0.022937205620110035)
     | > loss_mel_ce: 3.5183804035186768  (3.3911362886428833)
     | > loss: 3.5427913665771484  (3.414073467254638

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.01636296510696411 (+0.0005138516426086426)
     | > avg_loss_text_ce: 0.02352036675438285 (-6.957584992051125e-05)
     | > avg_loss_mel_ce: 3.3293262720108032 (-0.015247464179992676)
     | > avg_loss: 3.352846622467041 (-0.015317022800445557)

 > BEST MODEL : ./training_outputs/xttsv2_finetune_20250504_1250-May-04-2025_12+50PM-ca1939c\best_model_2070.pth
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


EvalStats/avg_loader_time,█▁▃▂▃▃▁▃▃
EvalStats/avg_loss,█▅▃▃▂▂▂▁▁
EvalStats/avg_loss_mel_ce,█▅▃▃▂▂▂▁▁
EvalStats/avg_loss_text_ce,█▇▇▆▅▄▃▂▁
TrainEpochStats/avg_grad_norm,▁▁▁▁▁▁▁▁
TrainEpochStats/avg_loader_time,█▄▂▂▃▂▄▁
TrainEpochStats/avg_loss,█▅▄▃▂▂▁▁
TrainEpochStats/avg_loss_mel_ce,█▅▄▃▂▂▁▁
TrainEpochStats/avg_loss_text_ce,█▆▆▃▄▄▃▁
TrainEpochStats/avg_step_time,▅▂▁▆▂█▂▁
TrainEpochStats/epoch_time,▁▂▂▆▄█▃▃
